In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [2]:
device = 'my_model' if torch.cuda.is_available() else 'cpu'
print(device)

## hyperperemeter

### function: get_batch()
block_size = 8 ### at most 6 character in a block
batch_size = 4

## function: for optimizing the model
max_iters = 10000
learning_rate = 3e-4

##
eval_iters = 250
# dropout = 0.2

cpu


In [3]:
with open('rembrandt_van_rijn.txt', 'r', encoding='utf-8') as f:
    text = f.read()
#print(text[:200]) 

## unique characters of this text
chars = sorted(set(text))
print(chars)


vocab_size = len(chars)

['\n', ' ', '"', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '=', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '|', 'Æ', 'È', 'É', 'Ö', '×', 'Ü', 'ä', 'æ', 'ç', 'è', 'é', 'ï', 'ò', 'ö', 'ü', '\ufeff']


In [4]:
string_to_int = {char:i for i, char in enumerate(chars)} ## each charcater is mapped into corrresponding index {key: value} --- dictionary { 'N': 40 }
int_to_string = {i:char for i, char in enumerate(chars)} ##  each index is mapped into corresponding character --- dictionary  { 74: 's' }

## lambda function named encode and decode
encode = lambda string: [string_to_int[char] for char in string] ##  list comprehension that iterates over each character c in the input string s
## i/p = my, encode = [3,10] (m = 3, y = 10)

decode = lambda l: ''.join([int_to_string[i] for i in l]) ## 
## i/p l = [[3, 10], [10,3] , .....] o/p = my ym

data = torch.tensor(encode(text), dtype = torch.long) ## each word will be as integer number
print(data)

tensor([98, 44, 31,  ...,  0,  0,  0])


In [5]:
n = int(0.8 * len(data)) ## get the size of 80% of the data for seperating it for train and test

train_data = data[:n]
validation_data = data[n:]

def get_batch(split):
    data = train_data if split == 'train' else validation_data
    # block_size = 8 ##  declared upper ---- here, only for remembaring
    # batch_size = 4
    ix = torch.randint(len(data) - block_size, (batch_size,)) # generate random indices from 0 to len(data) - block_size, batch_size x _ dimension
    # print(ix) # tensor([60, 60,  1, 70, 61,  0, 71, 73])
    x = torch.stack([data[i:i + block_size] for i in ix]) ## construction of input sequence --- if [60, 60,  1, 70, 61,  0, 71, 73]
    y = torch.stack([data[i + 1: i + block_size+1] for i in ix]) ## construction of output sequence --- then [60,  1, 70, 61,  0, 71, 73, 70]
    
    x, y = x.to(device), y.to(device) ##  to use GPU for better preformance
    return x, y

In [6]:
@torch.no_grad() ##  pytorch does not use gredient for fast computation
def loss_estimation(): ## called from the pytorch optimizer cell
    out = {}
    model.eval() ##  for evaluating the model
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean() ## {train: 2.123}
    model.train() ### train the model
    return out

In [ ]:
## define and call model 
class GPT_language_model(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)
        
    ## why is it important to write forward pass function in pytorch for beginners
    def forward(self, index, targets=None): ## input index -- token indecies
        logits = self.token_embedding_table(index)

        if targets is None:
            loss = None
            
        else:
            B, T, C = logits.shape 
            ## batch size(sequences processed prallely), 
            ## sequence length (represent length of each sequence),  
            ## number of classes (vocabulary size)

            ## model outputs logits in a typical sequence prediction task
            logits = logits.view(B * T, C) ## logit tensor usually has this shape size -- we have to reshape it 
            targets = targets.view(B * T) ## no need to reshape as ligits, since its structure is correct for computing the loss 
            loss = F.cross_entropy(logits, targets) ## https://pytorch.org/docs/stable/generated/torch.nn.functional.cross_entropy.html#torch.nn.functional.cross_entropy
        
        return logits, loss


    def generate(self, index, max_new_tokens):
        '''
        index : current index of the context
        depend on the current index context it will generate next 500 characters index , then using decode we get the actual characters
        '''

        for _ in range(max_new_tokens):

            # prediction -- to get the logits for the next token given the current context 'index'
            logits, loss = self.forward(index) ## method to get the logits for the next token given the current context 'index'.
            # last timestemp
            logits = logits[:, -1, :] # becomes (B, C) ---- logit[B*T:C]---- This selects the logits corresponding to predicting the next token after the current context.
            # apply softmax to get probabilities --- form of normalizations
            probs = F.softmax(logits, dim = -1) # (B, C) --- This converts the logits into a probability distribution over the vocabulary
            # sample from the distribution
            index_next = torch.multinomial(probs, num_samples = 1) # (B, 1) ---  It samples from the probability distribution to select the index of the next token
            # appened sample index to the running sequence
            index = torch.cat((index, index_next), dim = 1) # (B, T + 1)
        return index

### object for model class
model = bigram_language_model(vocab_size)
m = model.to(device)

In [ ]:
## pytorch optimizer
optimizer = torch.optim.AdamW(model.parameters(),  lr = learning_rate)

for iter in range(max_iters):
    if iter % eval_iters == 0:
        losses = loss_estimation()
        print(F'step: {iter}, train loss: {losses["train"]:.4f}, validation loss: {losses["val"]:.4f}')
    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model.forward(xb, yb) ## forward pass
    optimizer.zero_grad(set_to_none = True) # gradient
    loss.backward() ## backward pass
    optimizer.step() ## take the optimized step
print(loss.item())